In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import OAI
import json
from datetime import datetime

In [5]:
h = OAI.Helper("seeds_ask", "./cache")
f = OAI.askFCT("seeds_ask", "./cache")
h.GOTOCACHE = "./cache/"
f.GOTOCACHE = "./cache/"


In [6]:
import pandas as  pd

df = pd.read_parquet('data/articles.parquet.gzip')

In [7]:
df

,file_name,content,LEN
0,4c0323220b8e42ef6b79dd4d720ce80a,Enshittification isn’t caused by venture capit...,13140
1,397a8902bff1e80356fc872c0a0f5d98,Cultivating A Future in Love: Meet Our Microgr...,8551
2,6f7b4e8be445b769c44531f844aabe0f,Trump 'waved a white flag to Chinese hackers' ...,6017
3,72ad2ad82a4064cc0219178869779cd8,"Jevons paradox\nIn economics, the Jevons parad...",21035
4,044d2eabf669c7b4229e5803daf2e757,Generative artificial intelligence output base...,5208
...,...,...,...
3175,e8b7faf708568f9f39abe04b778c4631,I would say my efficiency is up ~20% since sta...,14337
3176,b79a2baa87b68283198416791b93bce4,The U.S. government has restricted sales of Nv...,2125
3177,d89d7cc690ceaaafaecc9f40ff8230c5,My kids and I just played D&D with ChatGPT4 as...,34489
3178,69637dcd83c48ebde0610a61a27b1989,The newest Kindle is the first truly new Kindl...,6249


In [8]:
from fcts import getWeaksignals, getEmergingIssues, getEmergingBehaviors, getEmergingTechnologies

WSFct = getWeaksignals()
Issues = getEmergingIssues()
Techs = getEmergingTechnologies()
Behaviors = getEmergingBehaviors()

In [9]:
txt = df["content"].iloc[0]

In [10]:
def getEmergingIssues(txt):

    messages, chat_response = f.askFct(
        "Find the emerging issues in the following text\n\n" ,
        txt.strip(),
        Issues,
        modelGPT="gpt-4o-mini",
        ow=False,
        src="signalsweaks",
        seed=".",
    )
    x = messages[-1]["function_call"]["arguments"]
    df = pd.DataFrame(json.loads(x)["issues"])

    return df

In [11]:
def getWeakSignals(txt):

    messages, chat_response = f.askFct(
        "Find the weak signals in the following text\n\n" ,
        txt.strip(),
        WSFct,
        modelGPT="gpt-4o-mini",
        ow=False,
        src="signalsweaks",
        seed=".",
    )
    x = messages[-1]["function_call"]["arguments"]
    df = pd.DataFrame(json.loads(x)["signals"])

    return df

In [12]:
def getTech(txt):

    messages, chat_response = f.askFct(
        "Find the emerging technologies in the following text\n\n" ,
        txt.strip(),
        Techs,
        modelGPT="gpt-4o-mini",
        ow=False,
        src="signalsweaks",
        seed=".",
    )
    x = messages[-1]["function_call"]["arguments"]
    df = pd.DataFrame(json.loads(x)["technology"])

    return df

In [13]:
def getBehaviors(txt,ow=False,seed="."):

    messages, chat_response = f.askFct(
        "Find the emerging behaviors in the following text\n\n" ,
        txt.strip(),
        Behaviors,
        modelGPT="gpt-4o-mini",
        ow=ow,
        src="signalsweaks",
        seed=seed,
    )
    x = messages[-1]["function_call"]["arguments"]
    df = pd.DataFrame(json.loads(x)["behaviors"])

    return df

In [14]:
done = list(pd.read_parquet("data/emergingissues.parquet.gzip").id.unique())
done

['4c0323220b8e42ef6b79dd4d720ce80a',
 '397a8902bff1e80356fc872c0a0f5d98',
 '6f7b4e8be445b769c44531f844aabe0f',
 '72ad2ad82a4064cc0219178869779cd8',
 '044d2eabf669c7b4229e5803daf2e757',
 'fcae2605790df6386dcc1465cb294d83',
 '3b889be2e955d68a002b96b66f790650',
 'b8ae24e6d0683d0069e0ee250a9d21f0',
 '89c33ff05c7560194e8902922720afa5',
 '7497f712012e964266a440f7ff73974e',
 'c0c7f3a02e89c04a227c3a467c10da68',
 'a9a73fc2bc55a2543c19a893b64fd671',
 '46ebbec03ee7d64ca3a457cb2882ba27',
 '299248e05dbfb0ab57ae84fc170ee39a',
 'fb7bff9fd47bc5ee71e96591add37ae8',
 'b87b3329ddb8dd4710c7176258a84ff3',
 '5760974f784e5f86908c8c3e16b57971',
 '83cfb6d5b04806b53a738ede2cb70246',
 '6904cf7cabc6d7203f72508bbc5b9575',
 '7c4e3bc9d5d97f1002eb9e002ccda1c8',
 '9786ba9f5842c13525de5cff9ee70d53',
 'd06afe4260c7035e564077936d8556ff',
 '32310a2097cecfb2b52a21a7c16fa9e8',
 '814f676017307cba8b54213432a333bc',
 '22b9f4fbbb1a4134de2f8a3412b08393',
 '742d65c9012edd7ca019583896e72b22',
 'cd6a3016e9c9fa46f13d29ded245803a',
 

In [ ]:
W,X,Y,Z = [],[],[],[]
for ix, row in df.iterrows():
    current_dateTime = datetime.now()
    if ix < 30000:
        txt = row["content"]
        if not ix%10:
            print(ix)
        
        try:
            w = getEmergingIssues(txt)
            w["id"] = row["file_name"]
            w["datetime"] = current_dateTime
            W.append(w)
        except:
            pass

        try:
            x = getWeakSignals(txt)
            x["id"] = row["file_name"]
            x["datetime"] = current_dateTime
            X.append(x)
        except:
            pass

        try:
            y = getTech(txt)
            y["id"] = row["file_name"]
            y["datetime"] = current_dateTime
            Y.append(y)
        except:
            pass
        
        try:
            z = getBehaviors(txt)
            z["id"] = row["file_name"]
            z["datetime"] = current_dateTime
            Z.append(z)
        except:
            pass

W = pd.concat(W).reset_index(drop=True)
X = pd.concat(X).reset_index(drop=True)
Y = pd.concat(Y).reset_index(drop=True)
Z = pd.concat(Z).reset_index(drop=True)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
{'error': {'message': "This model's maximum context length is 128000 tokens. However, your messages resulted in 181669 tokens (181489 in the messages, 180 in the functions). Please reduce the length of the messages or functions.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
1190
1200
1210
1220
1230
1240
1250
{'error': {'message': "This model's maximum context length is 128000 tokens. However, your messages resulted in 225150 tokens (224970 in the messages, 180 in th

In [16]:
W.to_parquet("data/emergingissues.parquet.gzip",compression="gzip")
X.to_parquet("data/weaksignals_v2.parquet.gzip",compression="gzip")
Y.to_parquet("data/technologies.parquet.gzip",compression="gzip")
Z.to_parquet("data/behaviors.parquet.gzip",compression="gzip")

In [17]:
print(len(W),len(X),len(Y),len(Z))

12443 9736 9016 12319
